RE-RANKING

In [1]:
!pip install -U beir
!pip install 'elasticsearch<7.14.0'

     |████████████████████████████████| 52 kB 891 kB/s 
     |████████████████████████████████| 78 kB 9.6 MB/s 
     |████████████████████████████████| 8.4 MB 17.6 MB/s 
     |████████████████████████████████| 378 kB 72.2 MB/s 
     |████████████████████████████████| 4.4 MB 56.8 MB/s 
     |████████████████████████████████| 3.1 MB 57.6 MB/s 
     |████████████████████████████████| 3.3 MB 59.7 MB/s 
     |████████████████████████████████| 1.2 MB 57.1 MB/s 
     |████████████████████████████████| 59 kB 9.1 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
     |████████████████████████████████| 895 kB 67.1 MB/s 
  Created wheel for beir: filename=beir-0.2.3-py3-none-any.whl size=45805 sha256=ece0d33f8145933bc87f567cc670b09c57ffc5f060c59a9430e6fb4db1e604e4
  Stored in directory: /root/.cache/pip/wheels/af/29/4c/261c842753d34921b8b35c1b8679720114f8536654767430dd
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=265119 sha256=2038f9b20e

In [2]:
import pathlib, os
import time
import pandas as pd
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.reranking.models import CrossEncoder
from beir.reranking import Rerank

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
if not os.path.exists('elasticsearch-oss-7.9.2-linux-x86_64.tar.gz'):
  !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
  !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
  !tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
  !sudo chown -R daemon:daemon elasticsearch-7.9.2/
  !shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

--2021-11-06 11:13:17--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229941304 (219M) [application/x-gzip]
Saving to: ‘elasticsearch-oss-7.9.2-linux-x86_64.tar.gz’

elasticsearch-oss-7 100%[===================>] 219.29M  9.24MB/s    in 24s     

2021-11-06 11:13:41 (9.01 MB/s) - ‘elasticsearch-oss-7.9.2-linux-x86_64.tar.gz’ saved [229941304/229941304]

--2021-11-06 11:13:41--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 20

In [4]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


In [5]:
time.sleep(20)

In [6]:
!ps -ef | grep elasticsearch

root         403     401  0 11:13 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       404     403 79 11:13 ?        00:00:15 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-1155147084120170201 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecou

In [7]:
!curl -sX GET "localhost:9200/"

{
  "name" : "2d7fd9f63853",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "ST6JFSg2QUeO_wmm3b99tg",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [8]:
def retrieve_lexical(dataset):
  hostname = 'localhost' 
  url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'
  out_dir = os.path.join(os.getcwd(), 'datasets')
  dataset_r = dataset
  index_r = dataset_r
  data_path_r = util.download_and_unzip(url.format(dataset_r), out_dir)
  corpus_r, queries_r, qrels_r = GenericDataLoader(data_path_r).load(split='test') # 'test', 'train', 'dev'
  model_r = BM25(index_name=index_r, hostname=hostname, initialize=True) # initialize=True : reindex
  retriever_r = EvaluateRetrieval(model_r)
  results_r = retriever_r.retrieve(corpus_r, queries_r)
  return corpus_r, queries_r, qrels_r, retriever_r, results_r

In [9]:
def rerank_model(reranker, corpus, queries, qrels, retriever, results, top_k):
  rerank_results = reranker.rerank(corpus, queries, results, top_k=top_k)
  return EvaluateRetrieval.evaluate(qrels, rerank_results, retriever.k_values)

In [10]:
def evaluate(name, ndcg, _map, recall, precision):
  c_map = 'MAP@10'
  c_map_h = 'MAP@100'
  c_map_t = 'MAP@1000'
  c_ndcg = 'NDCG@10'
  c_ndcg_h = 'NDCG@100'
  c_ndcg_t = 'NDCG@1000'
  c_pre = 'P@10'
  c_pre_h = 'P@100'
  c_pre_t = 'P@1000'
  c_rec = 'Recall@10'
  c_rec_h = 'Recall@100'
  c_rec_t = 'Recall@1000'
  eval_dict = {
      c_map: [_map[c_map]], 
      c_map_h: [_map[c_map_h]],
      c_map_t: [_map[c_map_t]],
      c_ndcg: [ndcg[c_ndcg]], 
      c_ndcg_h: [ndcg[c_ndcg_h]],
      c_ndcg_t: [ndcg[c_ndcg_t]],
      c_pre: [precision[c_pre]], 
      c_pre_h: [precision[c_pre_h]],
      c_pre_t: [precision[c_pre_t]],
      c_rec: [recall[c_rec]],
      c_rec_h: [recall[c_rec_h]],
      c_rec_t: [recall[c_rec_t]]
      }
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [name]
  return eval_df

In [11]:
def evaluate_rerank(rerank, corpus, queries, qrels, retriever, results, top_k):
  r_ndcg, r_map, r_recall, r_precision = rerank_model(rerank['rr'], corpus, queries, qrels, retriever, results, top_k)
  return evaluate(rerank['name'], r_ndcg, r_map, r_recall, r_precision)

In [12]:
def run_all_models(models, corpus, queries, qrels, retriever, results, top_k):
  rr_evals = []
  for model in models:
      rr_evals.append(evaluate_rerank(model, corpus, queries, qrels, retriever, results, top_k))
  return rr_evals

In [13]:
ag_d = 'arguana'
fq_d = 'fiqa'
ag_corpus, ag_queries, ag_qrels, ag_retriever, ag_results = retrieve_lexical(ag_d)
ag_ndcg, ag_map, ag_recall, ag_precision = ag_retriever.evaluate(ag_qrels, ag_results, ag_retriever.k_values)
fq_corpus, fq_queries, fq_qrels, fq_retriever, fq_results = retrieve_lexical(fq_d)
fq_ndcg, fq_map, fq_recall, fq_precision = fq_retriever.evaluate(fq_qrels, fq_results, fq_retriever.k_values)

/content/datasets/arguana.zip:   0%|          | 0.00/3.60M [00:00<?, ?iB/s]

  0%|          | 0/8674 [00:00<?, ?it/s]

que: 100%|██████████| 11/11 [01:25<00:00,  7.77s/it]


/content/datasets/fiqa.zip:   0%|          | 0.00/17.1M [00:00<?, ?iB/s]

  0%|          | 0/57638 [00:00<?, ?it/s]

que: 100%|██████████| 6/6 [00:33<00:00,  5.64s/it]


In [14]:
lexical_eval = []
lexical_eval.append(evaluate(ag_d, ag_ndcg, ag_map, ag_recall, ag_precision))
lexical_eval.append(evaluate(fq_d, fq_ndcg, fq_map, fq_recall, fq_precision))

In [15]:
batch_size = 128
top_k = 100
cmtb = 'cross-encoder/ms-marco-TinyBERT-L-2-v2'
cmlm = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
cmeb = 'cross-encoder/ms-marco-electra-base'
rr_cmtb = {'name': cmtb, 'rr': Rerank(CrossEncoder(cmtb), batch_size=batch_size)}
rr_cmlm = {'name': cmlm, 'rr': Rerank(CrossEncoder(cmlm), batch_size=batch_size)}
rr_cmeb = {'name': cmeb, 'rr': Rerank(CrossEncoder(cmeb), batch_size=batch_size)}
models = [rr_cmtb, rr_cmlm, rr_cmeb]
ag_res = run_all_models(models, ag_corpus, ag_queries, ag_qrels, ag_retriever, ag_results, top_k)
fq_res = run_all_models(models, fq_corpus, fq_queries, fq_qrels, fq_retriever, fq_results, top_k)

DEBUG:filelock:Attempting to acquire lock 139678732774992 on /root/.cache/huggingface/transformers/6b680f9419e5966765fbb1fa92e50daa731d68dff0b3d74cc3cf6e017e4a38f6.1f6f426ec040d209612d0d4625bffd38549004f8a0a7df44e78e6157bbe2e462.lock
DEBUG:filelock:Lock 139678732774992 acquired on /root/.cache/huggingface/transformers/6b680f9419e5966765fbb1fa92e50daa731d68dff0b3d74cc3cf6e017e4a38f6.1f6f426ec040d209612d0d4625bffd38549004f8a0a7df44e78e6157bbe2e462.lock


Downloading:   0%|          | 0.00/787 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139678732774992 on /root/.cache/huggingface/transformers/6b680f9419e5966765fbb1fa92e50daa731d68dff0b3d74cc3cf6e017e4a38f6.1f6f426ec040d209612d0d4625bffd38549004f8a0a7df44e78e6157bbe2e462.lock
DEBUG:filelock:Lock 139678732774992 released on /root/.cache/huggingface/transformers/6b680f9419e5966765fbb1fa92e50daa731d68dff0b3d74cc3cf6e017e4a38f6.1f6f426ec040d209612d0d4625bffd38549004f8a0a7df44e78e6157bbe2e462.lock
DEBUG:filelock:Attempting to acquire lock 139678742878800 on /root/.cache/huggingface/transformers/bfd3f7b24e7051275a86bebcf2cb3b982287862bf06023b76409efa5cf56bdd1.13465d11a8eb2056874a62c3ff0c97f17c4204bd6cef1d158a305ea9d9d2b826.lock
DEBUG:filelock:Lock 139678742878800 acquired on /root/.cache/huggingface/transformers/bfd3f7b24e7051275a86bebcf2cb3b982287862bf06023b76409efa5cf56bdd1.13465d11a8eb2056874a62c3ff0c97f17c4204bd6cef1d158a305ea9d9d2b826.lock


Downloading:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139678742878800 on /root/.cache/huggingface/transformers/bfd3f7b24e7051275a86bebcf2cb3b982287862bf06023b76409efa5cf56bdd1.13465d11a8eb2056874a62c3ff0c97f17c4204bd6cef1d158a305ea9d9d2b826.lock
DEBUG:filelock:Lock 139678742878800 released on /root/.cache/huggingface/transformers/bfd3f7b24e7051275a86bebcf2cb3b982287862bf06023b76409efa5cf56bdd1.13465d11a8eb2056874a62c3ff0c97f17c4204bd6cef1d158a305ea9d9d2b826.lock
DEBUG:filelock:Attempting to acquire lock 139678740221904 on /root/.cache/huggingface/transformers/582d53ff7c43533bb421f3d49d2d9a29e7fedf2318e5180cdca3539af5401485.4b5f6d66d38b524b7f32e4e697332bb95cd2908b0534a288fc926d6313725344.lock
DEBUG:filelock:Lock 139678740221904 acquired on /root/.cache/huggingface/transformers/582d53ff7c43533bb421f3d49d2d9a29e7fedf2318e5180cdca3539af5401485.4b5f6d66d38b524b7f32e4e697332bb95cd2908b0534a288fc926d6313725344.lock


...

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

Batches:   0%|          | 0/507 [00:00<?, ?it/s]

Batches:   0%|          | 0/507 [00:00<?, ?it/s]

Batches:   0%|          | 0/507 [00:00<?, ?it/s]



---



LEXICAL SEARCH (BM25)

In [16]:
pd.concat(lexical_eval)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
arguana,0.38394,0.39441,0.39462,0.47163,0.51669,0.52185,0.07532,0.00952,0.00099,0.7532,0.95164,0.99147
fiqa,0.19102,0.20421,0.20590,0.25365,0.31256,0.34704,0.07022,0.01304,0.00192,0.3244,0.54731,0.75728




---



**ARGUANA** - [Homepage](http://argumentation.bplaced.net/arguana/data) - [Paper](https://aclanthology.org/P18-1023.pdf)

In [17]:
pd.concat(ag_res)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
cross-encoder/ms-marco-TinyBERT-L-2-v2,0.26658,0.28248,0.28248,0.33971,0.41887,0.41887,0.05775,0.00952,0.00095,0.57752,0.95164,0.95164
cross-encoder/ms-marco-MiniLM-L-6-v2,0.33309,0.34678,0.34678,0.41573,0.47608,0.47608,0.06842,0.00952,0.00095,0.68421,0.95164,0.95164
cross-encoder/ms-marco-electra-base,0.24320,0.26134,0.26134,0.31054,0.40038,0.40038,0.05306,0.00952,0.00095,0.53058,0.95164,0.95164


**FIQA** - [Homepage](https://sites.google.com/view/fiqa/home) - [Paper](https://www.researchgate.net/publication/324629350_WWW'18_Open_Challenge_Financial_Opinion_Mining_and_Question_Answering)

In [18]:
pd.concat(fq_res)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
cross-encoder/ms-marco-TinyBERT-L-2-v2,0.21211,0.22587,0.22587,0.27922,0.33386,0.33386,0.07623,0.01304,0.0013,0.34885,0.54731,0.54731
cross-encoder/ms-marco-MiniLM-L-6-v2,0.27447,0.28427,0.28427,0.34725,0.38425,0.38425,0.09429,0.01304,0.0013,0.41330,0.54731,0.54731
cross-encoder/ms-marco-electra-base,0.27759,0.28712,0.28712,0.34904,0.38641,0.38641,0.09491,0.01304,0.0013,0.40889,0.54731,0.54731


end of fun.